In [1]:
import pandas as pd
from collections import Counter
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
# data seems to be lines of json rather than a single object
# loading it into a list so I can carry out merging this way
# only feasible when the data size is small. Wouldn't do this if data size was huge
d = []
with open('../Data/customers.json') as f:
    for line in f:
        d.append(json.loads(line))

In [4]:
# take the first data point and turn it into a dataframe, other data points can then append on this root frame
# (again, it would've been more efficient if this was done on the fly when reading lines off json file 
# but this method in my opinion is more readable)

ordersFrame = pd.DataFrame(d[0]['orders'])
customers_frame = pd.json_normalize(d[0]['customer']) 
ordersFrame['joinkey'], customers_frame['joinkey'] = 0,0 # temporary join key for customer and orders
df = pd.merge(ordersFrame,customers_frame, on='joinkey').drop(columns='joinkey')
df['fraudulent'] = d[0]['fraudulent']

In [5]:
# repeat the above process for all line of json
for i in range(1,len(d)):
    orders_frame = pd.DataFrame(d[i]['orders'])
    customers_frame = pd.json_normalize(d[i]['customer']) 
    orders_frame['joinkey'], customers_frame['joinkey'] = 0,0
    merged_orders_customers = pd.merge(orders_frame,customers_frame, on='joinkey').drop(columns='joinkey')
    merged_orders_customers['fraudulent'] = d[i]['fraudulent']
    df = pd.concat([df, merged_orders_customers], ignore_index=True)

In [6]:
# join the rest of the data on foreign keys
trans = pd.json_normalize(data=d, record_path="transactions")
df = pd.merge(df, trans, on="orderId")
payments = pd.json_normalize(data=d, record_path="paymentMethods")
df = pd.merge(df, payments, on="paymentMethodId")

In [8]:
df.head(5).T

,0,1,2,3,4
orderId,vjbdvd,yp6x27,nlghpa,uw0eeb,bn44oh
orderAmount,18,26,45,23,43
orderState,pending,fulfilled,fulfilled,fulfilled,fulfilled
orderShippingAddress,"5493 Jones Islands\nBrownside, CA 51896","5493 Jones Islands\nBrownside, CA 51896","898 Henry Ports\nNew Keithview, CA 95893-2497",356 Elizabeth Inlet Suite 120\nPort Joshuabury...,"5093 Bryan Forks\nJoshuaton, FM 01565-9801"
customerEmail,josephhoward@yahoo.com,josephhoward@yahoo.com,evansjeffery@yahoo.com,evansjeffery@yahoo.com,evansjeffery@yahoo.com
customerPhone,400-108-5415,400-108-5415,1-788-091-7546,1-788-091-7546,1-788-091-7546
customerDevice,yyeiaxpltf82440jnb3v,yyeiaxpltf82440jnb3v,r0jpm7xaeqqa3kr6mzum,r0jpm7xaeqqa3kr6mzum,r0jpm7xaeqqa3kr6mzum
customerIPAddress,8.129.104.40,8.129.104.40,219.173.211.202,219.173.211.202,219.173.211.202
customerBillingAddress,"5493 Jones Islands\nBrownside, CA 51896","5493 Jones Islands\nBrownside, CA 51896",356 Elizabeth Inlet Suite 120\nPort Joshuabury...,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,356 Elizabeth Inlet Suite 120\nPort Joshuabury...
fraudulent,False,False,True,True,True
